# _TriScale_ - Experiment Sizing

> This notebook is intended for **live tutorial** sessions about _TriScale._  
Here is the [self-study version]().

To get started, we need to import a few Python modules. All the _TriScale_-specific functions are part of one module called `triscale`.

In [ ]:
import os
from pathlib import Path

import pandas as pd
import numpy as np

import triscale

## Basics

_TriScale_'s `experiment_sizing()` function implements the computation of the minimal number of samples required to estimate any percentile with any confidence level.

In [ ]:
percentile = 50 # the median
confidence = 95 # the confidence level, in %

triscale.experiment_sizing(
    percentile, 
    confidence,
    verbose=True); 

We can change the values in the cell above to see how the number of samples evolves with a larger confidence level or more extreme percentiles. 

Note that the probability distributions are symetric: it takes the same number of samples to compute a lower bound for the $p$-th  percentile as to compute an upper bound for the $(1-p)$-th percentile.

In [ ]:
percentile = 20 # the median
confidence = 95 # the confidence level, in %

if (triscale.experiment_sizing(percentile,confidence) == 
    triscale.experiment_sizing(100-percentile,confidence)):
    print("It takes the same number of samples to estimate \
the \n{}-th and \n{}-th percentiles.".format(percentile, 100-percentile))


In [ ]:
# Sets of percentiles and confidence levels to try
percentiles = [0.1, 1, 5, 10, 25, 50, 75, 90, 95, 99, 99.9]
confidences = [75, 90, 95, 99, 99.9, 99.99]

# Computing the minimum number of runs for each (perc., conf.) pair
min_number_samples = []
for c in confidences:
    tmp = []
    for p in percentiles:
        N = triscale.experiment_sizing(p,c)
        tmp.append(N[0])
    min_number_samples.append(tmp)
    
# Put the results in a DataFrame for a convenient display of the results
df = pd.DataFrame(columns=percentiles, data=min_number_samples)
df['Confidence level'] = confidences
df.set_index('Confidence level', inplace=True)

display(df)

## Excluding outliers

By default, `experiment_sizing()` returns the minimal number of samples, such that the __smallest one__ is the percentile estimate (or the largerest one, if the percentile is $> 50$).

If the experiment is subject to outliers, or more generally to obtain tighter bounds, one may want to collect more samples. But how many? You can use the `robustness` argument to find out:

In [ ]:
percentile = 10
confidence = 99
triscale.experiment_sizing(
    percentile, 
    confidence,
    robustness=3,
    verbose=True); 

## Your turn: time to practice

Based on the explanations above, use _TriScale_'s `experiment_sizing` function to answer the following questions:
- What is the minimal number of runs required to estimate the
    - **90th** percentile with **90%** confidence?
    - **90th** percentile with **95%** confidence?
    - **95th** percentile with **90%** confidence?
- Based on the answers to the previous questions, is it harder (i.e., does it require more runs) to increase the confidence level, or to estimate a more extreme percentile? 

_Optional question (harder):_ 
- For $N = 50$ samples, what is the index $m$ of the best possible (i.e., the largest) lower bound for the 25th percentile, estimated with a 95% confidence level? 

In [ ]:
########## YOUR CODE HERE ###########
# ...
#####################################

### Solution

<details>
  <summary><br/>Click here show the solution.</summary>
  
```python
>>> print(triscale.experiment_sizing(90,90)[0])
22
>>> print(triscale.experiment_sizing(90,95)[0])
29
>>> print(triscale.experiment_sizing(95,90)[0])
45
```
We observe that it "costs" many more runs to estimate a more extreme percentile 
    (95th instead of 90th) than to increase the confidence level (90% to 95%). 
    This observation holds true in general. The number of runs required increases 
    exponentially when the percentiles get more extreme (close to $0$      or to $1$).
    
For the last question, we must play with the `robusteness` parameter. We can 
    write a simple loop to increase its value until the number of runs required  
    reaches 50.
    
```python
>>> r = 0
>>> while (triscale.experiment_sizing(25,95,r)[0] < 50):
>>>     r += 1 
>>> print(r)
7                                           
```        
Hence, we can exclude the 7 "worst" samples from the confidence interval. 
    With $N=50$ samples, the best lower bound for the 25th percentile with 95% confidence 
    is $x_8$     (assuming the first sample is $x_1$).
</details>

---
Next step: [Data Analysis](live_data-analysis.ipynb)  
[Back to repo](.)